In [ ]:
import neurogym as ngym
from neurogym import TrialEnv
import numpy as np

class AnnubesBlock(TrialEnv):
    # DEV: evaluate later on whether we want to keep this as a children of TrialEnv or implement it in the parent class
    # DEV: shuffle_trials functionality is missing
    # DEV: max_sequential functionality is missing
    # DEV: iti functionality is missing
    # EXP: n_outputs and output_behavior functionality is different from the annubes implementation, but I think it is better to keep it this way
    # DEV: scaling functionality is missing
    def __init__(self, session = {"v": 0.5, "a": 0.5}, stim_intensities = [0.8, 0.9, 1.0], stim_time = 1000, decision_time = 100, catch_prob = 0.5, fix_intensity = 0, fix_time = 500, dt = 20, tau = 100, n_outputs = 2, output_behavior = [0, 1], noise_std = 0.01, random_seed = None):
        super().__init__(dt=dt)

        self.session = session
        self.stim_intensities = stim_intensities
        self.stim_time = stim_time
        # EXP: define together the decision time for the different tasks
        self.decision_time = decision_time
        self.catch_prob = catch_prob
        self.fix_intensity = fix_intensity
        # DEV: add range for fix_time
        self.fix_time = fix_time
        self.dt = dt
        self.tau = tau
        self.n_outputs = n_outputs
        self.output_behavior = output_behavior
        self.noise_std = noise_std
        self.random_seed = random_seed
        alpha = dt / self.tau
        self.noise_factor = self.noise_std * np.sqrt(2 * alpha) / alpha
        # Set random state
        # DEV: improve how BaseEnv implements random_seed setting
        if random_seed is None:
            rng = np.random.default_rng(random_seed)
            random_seed = rng.integers(2**32)
        self._rng = np.random.default_rng(random_seed)
        self._random_seed = random_seed
        self.timing = {'fixation': self.fix_time, 'stimulus': self.stim_time, 'decision': self.decision_time}
        # Set the name of each input dimension
        obs_space_name = {'fixation': 0, 'start': 1, 'v': 2, 'a': 3}
        self.observation_space = ngym.spaces.Box(
            low = 0., high = 1., shape = (len(obs_space_name),), name = obs_space_name)
        # Set the name of each action value
        self.action_space = ngym.spaces.Discrete(self.n_outputs, name={'fixation': self.output_behavior[0], 'choice': self.output_behavior})

    def _new_trial(self):
        # Setting time periods and their order for this trial
        self.add_period(['fixation', 'stimulus', 'decision'])

        # Adding fixation and start signal values
        self.add_ob(self.fix_intensity, "fixation", where="fixation")
        self.add_ob(1, "stimulus", where="start")

        # Catch trial decision
        catch = self.rng.choice([0, 1], p=[self.catch_prob, 1 - self.catch_prob])
        if not catch:
            stim_type = self.rng.choice(list(self.session.keys()), p=list(self.session.values()))
            stim_value = self._rng.choice(self.stim_intensities, 1)

        for mod in self.session.keys():
            stim = stim_value if not catch and stim_type == mod else 0
            self.add_ob(stim, "stimulus", where=mod)

        if not catch:
            # DEV: verify that this way of adding noise is correct
            self.add_randn(0, self.noise_factor, 'stimulus', where=stim_type)

        # Set ground_truth
        groundtruth = 1 if not catch else 0
        self.set_groundtruth(groundtruth, period='decision', where='choice')

        # Trial information
        # DEV: add meaningful information to trial
        trial = {
            "ground_truth": groundtruth,

        }

        return trial

    def _step(self, action):
        # TODO exp: define the rewards looking at PerceptualDecisionMaking as guiding example
        # self.ob_now and self.gt_now correspond to
        # current step observation and groundtruth

        # If action equals to ground_truth, reward=1, otherwise 0
        reward = (action == self.gt_now) * 1.0

        terminated = False
        truncated = False
        # By default, the trial is not ended
        info = {'new_trial': False}
        return self.ob_now, reward, terminated, truncated, info

In [ ]:
env = AnnubesBlock()
print(env.timing)
print('----------------')
print(env.observation_space)
print(env.observation_space.name)
print('----------------')
print(env.action_space)
print(env.action_space.name)

In [ ]:
fig = ngym.utils.plot_env(
    env,
    ob_traces=['fixation', 'start', 'v', 'a'],
    num_trials=8,
)